In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from glob import glob
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from scipy import stats

In [89]:
mu_b = 5.7883818012 * 10**(-5)
mu_b

5.7883818012e-05

In [90]:
list_sweepVT = sorted(glob('*Voltage_Sweep.tsv.gz'))

In [91]:
list_calibration = glob('*.wlen_to_px.tsv.gz')

In [92]:
calibration = 1230.8 / (pd.read_csv(list_calibration[0], header=None).as_matrix().transpose())[0]

In [93]:
sweepVT_sigmam = np.array([pd.read_csv(file, sep='\t', header=None).as_matrix() for file in list_sweepVT[:9]])
sweepVT_sigmap = np.array([pd.read_csv(file, sep='\t', header=None).as_matrix() for file in list_sweepVT[9:]])

In [94]:
np.shape(sweepVT_sigmam)


(9, 81, 1340)

In [95]:
np.shape(sweepVT_sigmap)

(9, 81, 1340)

In [96]:
np.shape(sweepVT_sigmap[0,0,:])

(1340,)

In [97]:
spec = sweepVT_sigmap[0,15,:]
calibration

array([1.85623947, 1.85587281, 1.85550909, ..., 1.47222415, 1.47199701,
       1.47177171])

In [98]:
output_notebook()

Loading BokehJS ...

In [99]:
def lorentz(x, x_n, G):
    X = (x_n - x)/(G/2)
    return 1/(1+X**2)

In [100]:
a_min, a_max = 1.69, 1.72

In [101]:
def fit_a(data,
          cal=calibration,
          a_min=1.695,
          a_max=1.72,
          base_function=lorentz,
          p0=[1.70, 0.05]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(lorentz, cal_slice, data_slice, p0=p0)
    return popt[0]  

In [102]:
a_energies_sp = np.array([fit_a(field) for field in sweepVT_sigmap[:,27,:]])
a_energies_sm = np.array([fit_a(field) for field in sweepVT_sigmam[:,27,:]])
#g_factors = (a_energies_sm - a_energies_sp)/(mu_b * np.arange(9))
a_energies_sp

array([1.70597598, 1.70607911, 1.70624622, 1.7065384 , 1.70678533,
       1.70704057, 1.70738065, 1.70771908, 1.70853236])

In [103]:
p = figure(y_range=(1.705, 1.709))
xxx = np.arange(0,9,1)
p.scatter(xxx, a_energies_sm)
p.scatter(xxx, a_energies_sp, color='red')
show(p)

In [122]:
#a_energies_sp = np.array([fit_a(field) for field in sweepVT_sigmap[:,23,:]])
#a_energies_sm = np.array([fit_a(field) for field in sweepVT_sigmam[:,23,:]])
#g_factors = (a_energies_sm - a_energies_sp)/(mu_b * np.arange(9))
#a_energies_sp
a_energies_sp = np.array([fit_a(field) for field in sweepVT_sigmap[8,:,:]])
a_energies_sm = np.array([fit_a(field) for field in sweepVT_sigmam[8,:,:]])
#g_factors = (a_energies_sm - a_energies_sp)/(mu_b * np.arange(9))
a_energies_sp

/usr/lib/python3/dist-packages/scipy/optimize/minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


array([ 1.64393084e+00,  1.63988411e+00,  1.66214303e+00,  1.66496096e+00,
        1.65579595e+00,  1.63851251e+00,  1.61581697e+00, -1.74796531e+05,
       -5.79589143e+05, -9.00608835e+05, -7.38835577e+05, -5.67679631e+05,
        1.71147712e+00,  1.71024201e+00,  1.70943933e+00,  1.92377738e+00,
        1.99226229e+00,  1.70862409e+00,  2.10522136e+00,  1.70867166e+00,
        1.70864918e+00,  1.70855940e+00,  1.70846143e+00,  1.70855424e+00,
        1.70854637e+00,  1.70846544e+00,  1.70844795e+00,  1.70853236e+00,
        1.70854477e+00,  1.70846499e+00,  1.70847413e+00,  1.70870203e+00,
        1.70920274e+00,  1.70924333e+00,  1.70955573e+00,  1.70987077e+00,
        1.71026149e+00,  1.71064506e+00,  1.71073262e+00,  1.71116454e+00,
        1.71157690e+00,  1.71174626e+00,  1.71185399e+00,  1.71246844e+00,
       -8.62716846e+05, -1.21714594e+06, -8.02789969e+05, -5.04189247e+05,
       -8.22734865e+05, -8.88546547e+05, -9.58110177e+05, -9.32946185e+05,
       -5.50511535e+05, -

In [123]:
p = figure(y_range=(1.705, 1.71))
xxx = np.arange(0,81,1)
p.scatter(xxx, a_energies_sm)
p.scatter(xxx, a_energies_sp, color='red')
show(p)

In [106]:
p = figure()
slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(9) * mu_b, a_energies_sm)
teslas = np.arange(9) * mu_b
linear = slope * teslas + intercept
p.scatter(teslas, a_energies_sm)
p.line(mu_b * np.arange(9), linear, color='red')
print(slope)
#plt.plot(calibration, y)
show(p)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
sweepVT_sigmap[:,15,:]

In [ ]:
p = figure()
p.line(calibration, spec)
show(p)